In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
x,y,k=200,200,-1

In [3]:
capture=cv2.VideoCapture(0)

EVENT_LBUTTONDOWN Event below signifies left mouse button as an event,it overwrites the current x and y position

In [4]:
def take_input_images(action_event,x1,y1,flag,param):
    global x,y,k
    if action_event==cv2.EVENT_LBUTTONDOWN:
        x=x1
        y=y1
        k=1

In [5]:
cv2.namedWindow("Draw/Move_point")
cv2.setMouseCallback("Draw/Move_point",take_input_images)

In [6]:
while(True):
    _,input_image=capture.read()
    input_image=cv2.flip(input_image,1)
    grayscale_input_image=cv2.cvtColor(input_image,cv2.COLOR_BGR2GRAY)
    
    cv2.imshow("Draw/Move_point",input_image)
    if(k==1 or cv2.waitKey(30)==27):
        cv2.destroyAllWindows()
        break

KeyboardInterrupt: 

# optical flow(tracking) part

In [ ]:
stop=0

In [ ]:
old_points=np.array([[x,y]],dtype=np.float32).reshape(-1,1,2)

The below function uses the LUCAS KANADE OPTICAL FLOW METHOD PRESENT IN cv2

Mask is window size equal to the input image size it is blank,acts like a canvas

In [ ]:
mask=np.zeros_like(input_image)
while(True):
    _,new_input_image=capture.read()
    new_input_image=cv2.flip(new_input_image,1)
    new_grayscale_image=cv2.cvtColor(new_input_image,cv2.COLOR_BGR2GRAY)
    new_points,status,error=cv2.calcOpticalFlowPyrLK(grayscale_input_image,
                                                    new_grayscale_image,
                                                    old_points,
                                                    None,maxLevel=1,
                                                    criteria=(cv2.TERM_CRITERIA_EPS| cv2.TERM_CRITERIA_COUNT,15, 0.08))
    
    for i,j in zip(old_points,new_points):
        a,b=i.ravel()
        x,y=j.ravel()
        #j=[[1,2]].ravel()==>x=1,y=2
        
        if cv2.waitKey(2) & 0xff == ord('q'):
            stop = 1 #stopdrawing on q press
            
        elif cv2.waitKey(2) & 0xff == ord('w'):
            stop = 0 #start drawing on w press
        
        elif cv2.waitKey(2) == ord('n'):
            mask = np.zeros_like(new_input_image)
            #reset on pressing n
        
        #again start drawing
        if stop == 0:
            mask = cv2.line(mask, (a,b), (x,y), (0,0,255), 6)
        
        cv2.circle(new_input_image,(x,y),6,(0,255,0),-1)
        
    new_input_image=cv2.addWeighted(mask,0.3,new_input_image,0.7,0)
    cv2.putText(mask,"Press Q to quit W to draw N to reset",(10,50),
               cv2.FONT_HERSHEY_PLAIN,2,(255,255,255))
    
    cv2.imshow("Output",new_input_image)
    cv2.imshow("Drawing",mask)
    
    #use current image and store the old image
    gray_input_image=new_grayscale_image.copy()
    old_points=new_points.reshape(-1,1,2)
    
    if(cv2.waitKey(1) and 0xff==27):
        break

cv2.destroyAllWindows()
capture.release()